# WORKFLOW COMPLETE FOR GOOGLE COLAB

## Prepare Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
repo_path = "/content/drive/MyDrive/GitHub"

os.makedirs(repo_path, exist_ok=True)

# Move to folder
%cd {repo_path}

# Clone
!git clone https://github.com/jorgemasgomez/almondcv2.git

%cd {repo_path}/almondcv2

In [ ]:
#If you have cloned the repository previously simply move to the folder 
repo_path = "/content/drive/MyDrive/GitHub"
%cd {repo_path}/almondcv2

## Install requirements

In [ ]:
!pip install -r requirements_google_colab.txt #In Google Colab will be necessary to install it in each session

# 1. Pre-processing workflow

In [ ]:
#Import libraries
import os
from calibrations import  build_calibration, calibrate_color_and_distortion, calibrate_color, calibrate_distortion
from aux_functions import obtain_pixel_metric, ungroup_pic
from model_class import ModelSegmentation
import pandas as pd

#Set paths of the files
working_directory="/content/drive/MyDrive/database_almondcv2/"
chessboards=os.path.join(working_directory, "calibracion/chessboards") #folder with chessboard pitcures
raw_folder=os.path.join(working_directory,"pruebas_jorge")#folder with the pictures to calibrate
mtx_input_path=os.path.join(chessboards,"calibration_mtx.npz") #for distortion in npz format
standard_matrix_color=os.path.join(working_directory, "pruebas_jorge/28_10_CG-009.JPG") #picture of reference
output_calibrated=os.path.join(working_directory,"pruebas_jorge") #output folder for calibrated pictures

coin_model_path=os.path.join(working_directory,"models/coin_2022_yolov11_640.pt")
info_table=os.path.join(working_directory,"info_data.txt")

group_model_path=os.path.join(working_directory, "models/rectangle_2022_yolov11s_1280.pt")

## Color and Distortion calibration

In [ ]:
# First build your distortion model based in chessboards
build_calibration(chessboardSize=(6, 8), frameSize=(5472,3648), dir_path=chessboards, 
                  image_format=".jpg", size_of_chessboard_squares_mm=20)

In [ ]:
#Function for calibrate color and distortion
calibrate_color_and_distortion(raw_folder=raw_folder,mtx_input_path=mtx_input_path,output_calibrated=output_calibrated,
                                radius_param=10, standard_matrix=standard_matrix_color) #Standard matrix is a picture of reference to use instead of the original picture for error cases or simplicity

In [ ]:
#Function for calibrate distortion only

calibrate_distortion(input_folder=raw_folder, mtx_input=mtx_input_path, output_path=output_calibrated, input_picture=None)

In [ ]:
#Function for calibrate color only

calibrate_color(input_folder=raw_folder, output_path=output_calibrated,standard_matrix=standard_matrix_color,
                 force_standard_matrix="No")  #force_standard_matrix option uses in all the pictures the reference picture. In negative case, use only standard_matrix in error cases.

## Obtain pixel size

In [ ]:
#Deploy the model 

reference_model=ModelSegmentation(working_directory=working_directory)
contours_coin=reference_model.slice_predict_reconstruct(input_folder=output_calibrated, imgsz=640,
                                                         model_path=coin_model_path, slice_height=640, slice_width=640,
                                                         overlap_height_ratio=0.1, overlap_width_ratio=0.1,
                                                           retina_mask=True, conf=0.9)

In [ ]:
#Load info table
info_data_df=pd.read_csv(info_table,sep="\t")
# If we use a calibrated dataset but the info table was previous we can include CL_ automatically with this line
# info_data_df['Name_picture'] = info_data_df['Name_picture'].apply(lambda x: 'CL_' + x)

info_data_completed=obtain_pixel_metric(info_data=info_data_df, contours=contours_coin,
                                         output_directory=working_directory, reference=24.25) #reference in mm

## Ungroup pictures

In [ ]:
#Load group model
group_model=ModelSegmentation(working_directory=working_directory)
contours_groups=group_model.predict_model(model_path=group_model_path,
                               folder_input=output_calibrated,
                               imgsz=1280, check_result=False, max_det=2, retina_mask=False) #Retina mask not recommended here.

In [ ]:
#Obtain sample pictures and update info table. 
info_data_completed_path=os.path.join(working_directory, "info_data_completed_2022.txt")
info_data_completed=pd.read_csv(info_data_completed_path,sep="\t")

ungroup_pic(input_contours=contours_groups, output_path=working_directory, info_file=info_data_completed, axis="X") #axis indicate if the samples should be order according to Y or X axis

# 2. Develop your segmentation model

In [ ]:
#Import libraries
import os
from model_class import ModelSegmentation
from aux_functions import slicing
import cv2
#Inputs
working_directory="/content/drive/MyDrive/database_almondcv2/"
pictures_directory=os.path.join(working_directory, "kernel_2022")
pre_model=os.path.join(working_directory, "models/yolo11s-seg.pt")
output_directory=os.path.join(working_directory,"output_directory")

### Slicing

In [ ]:
#Slice_pictures for training
slicing(input_folder=pictures_directory,output_directory=working_directory,name_slicing="Slicing_kernel_2022", number_pictures=5, slice_height=320, slice_width=320)

### Training

In [ ]:
#Label with CVAT

zip_file=os.path.join(working_directory,"kernel_2022_320.zip")

In [ ]:
#Model segmentation training

model=ModelSegmentation(working_directory=working_directory)
model.train_segmentation_model(input_zip=zip_file, epochs=50,imgsz=320, name_segmentation="kernel_2022_320",
                                      pre_model=pre_model, batch=16, colab=True)

### Deploy and reconstruct a picture

#### Slice_predict_reconstruct approach

In [ ]:
# Join patches approach

model=ModelSegmentation(working_directory=working_directory)
masks=model.slice_predict_reconstruct(input_folder=pictures_directory,imgsz=320, model_path=model_path,
                                          slice_height=320, slice_width=320,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)

In [ ]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    cv2.imwrite(f"{output_directory}/{os.path.basename(mask[1])}", mask[0])


#### SAHI

In [ ]:
model=ModelSegmentation(working_directory=working_directory)
masks=model.predict_model_sahi(model_path=model_path, check_result=False, folder_input=pictures_directory,
                                            retina_masks=True,
                                              postprocess_match_threshold=0.2, overlap_height_ratio=0.2,
                                                overlap_width_ratio=0.2, postprocess_match_metric="IOS", 
                                                postprocess_type="GREEDYNMM", slice_height=320, slice_width=320,
                                                  confidence_treshold=0.8,
                                                  imgsz=320)

In [ ]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    mask[0].export_visuals(export_dir=output_directory, hide_labels=True, rect_th=1, file_name=f"{os.path.basename(mask[1])}")

# 3. Deploy your segmentation model

In [ ]:
#Import libraries
import os
from model_class import ModelSegmentation
import pickle
from pictures_class import Pictures
import pandas as pd
#Inputs
working_directory="/content/drive/MyDrive/database_almondcv2/"
pictures_directory=os.path.join(working_directory, "kernel_2022")
model_path=os.path.join(working_directory, "models/kernel_2022_yolov11s_320.pt")
info_data_completed_path=os.path.join(working_directory, "info_data_2022.txt")
info_data_completed_path_almond=os.path.join(working_directory, "info_data_2022.txt")
info_data_completed=pd.read_csv(info_data_completed_path,sep="\t")
info_data_completed_almond=pd.read_csv(info_data_completed_path_almond,sep="\t")

## Choose your reconstruction approach and measure (almond)

### Slice predict reconstruct

In [ ]:
# Join patches approach

model=ModelSegmentation(working_directory=working_directory)
masks=model.slice_predict_reconstruct(input_folder=pictures_directory,imgsz=320, model_path=model_path,
                                          slice_height=320, slice_width=320,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)

In [ ]:
## Example with slice predict reconstruct approach
pictures_object=Pictures(working_directory=working_directory, input_folder=pictures_directory,info_file=info_data_completed_almond,
                      fruit="kernel_almond", binary_masks=True, project_name="probando_watershed", blurring_binary_masks=False)
pictures_object.set_postsegmentation_parameters(sahi=False, segmentation_input=masks, smoothing=False, smoothing_iterations=2, kernel_smoothing=3,
                        watershed=True, kernel_watershed=5, threshold_watershed=0.6)
pictures_object.measure_almonds(margin=400)

# Save
with open(f'{working_directory}/pictures_object_watershed.pkl', 'wb') as file:
    pickle.dump(pictures_object, file)

### SAHI

In [ ]:
model=ModelSegmentation(working_directory=working_directory)
masks=model.predict_model_sahi(model_path=model_path, check_result=False, folder_input=pictures_directory,
                                            retina_masks=True,
                                              postprocess_match_threshold=0.2, overlap_height_ratio=0.2,
                                                overlap_width_ratio=0.2, postprocess_match_metric="IOS", 
                                                postprocess_type="GREEDYNMM", slice_height=320, slice_width=320,
                                                  confidence_treshold=0.8,
                                                  imgsz=320)

In [ ]:
## Example with SAHI approach
pictures_object=Pictures(working_directory=working_directory, input_folder=pictures_directory,info_file=info_data_completed_almond,
                      fruit="kernel_almond", binary_masks=True, project_name="probando",  blurring_binary_masks=False)
pictures_object.set_postsegmentation_parameters(sahi=True, segmentation_input=masks)
pictures_object.measure_almonds(margin=400)

# Guardar el objeto en un archivo
with open(f'{working_directory}/pictures_object_sahi.pkl', 'wb') as file:
    pickle.dump(pictures_object, file)

## Choose your reconstruction approach and measure (general)

### Slice predict reconstruct

In [ ]:
# Join patches approach

model=ModelSegmentation(working_directory=working_directory)
masks=model.slice_predict_reconstruct(input_folder=pictures_directory,imgsz=320, model_path=model_path,
                                          slice_height=320, slice_width=320,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)

In [ ]:
## Example with slice predict reconstruct approach
pictures_object=Pictures(working_directory=working_directory, input_folder=pictures_directory,info_file=info_data_completed,
                      fruit="kernel_almond", binary_masks=True, project_name="kernel_watershed", blurring_binary_masks=False)
pictures_object.set_postsegmentation_parameters(sahi=False, segmentation_input=masks, smoothing=False, smoothing_iterations=2, kernel_smoothing=3,
                        watershed=True, kernel_watershed=5, threshold_watershed=0.6)
pictures_object.measure_general(margin=400)

# Save
with open(f'{working_directory}/pictures_object_watershed.pkl', 'wb') as file:
    pickle.dump(pictures_object, file)

### SAHI

In [ ]:
model=ModelSegmentation(working_directory=working_directory)
masks=model.predict_model_sahi(model_path=model_path, check_result=False, folder_input=pictures_directory,
                                            retina_masks=True,
                                              postprocess_match_threshold=0.2, overlap_height_ratio=0.2,
                                                overlap_width_ratio=0.2, postprocess_match_metric="IOS", 
                                                postprocess_type="GREEDYNMM", slice_height=320, slice_width=320,
                                                  confidence_treshold=0.8,
                                                  imgsz=320)

In [ ]:
## Example with SAHI approach
pictures_object=Pictures(working_directory=working_directory, input_folder=pictures_directory,info_file=info_data_completed,
                      fruit="kernel_almond", binary_masks=True, project_name="kernel_2022_sahi",  blurring_binary_masks=False)
pictures_object.set_postsegmentation_parameters(sahi=True, segmentation_input=masks)
pictures_object.measure_general(margin=400)

# Guardar el objeto en un archivo
with open(f'{working_directory}/pictures_object_sahi.pkl', 'wb') as file:
    pickle.dump(pictures_object, file)

# 4. Morphometrics

In [ ]:
#Import libraries
import os
from morphometrics_functions import install_morphometrics_packages_r, exploratory_morphometrics_r, run_efourier_pca_morphometrics_r, run_plot_pca_morphometrics_r, run_kmeans_efourier_r, process_images_and_perform_pca

#Inputs

input_masks=r""
working_directory=r""



## Momocs

In [ ]:
# For installing libraries
install_morphometrics_packages_r()

In [ ]:
#For exploring the dataset
exploratory_morphometrics_r(info_data="", grouping_factor="", input_directory=input_masks,
                             output_directory=working_directory, show=True, nharmonics=10,nexamples=2)

In [ ]:
#For running EFA and PCA 
object_path=os.path.join(working_directory,"exploratory_plots","outlines_objects.rds")
run_efourier_pca_morphometrics_r(path_outline_objects=object_path, nharmonics=10, output_directory=working_directory,
                                  show=True, normalize="False", img_height_pca=1000, img_width_pca=1000)

In [ ]:
#For plotting PCA
object_path=os.path.join(working_directory,"efourier_results","pca_fourier.rds")
run_plot_pca_morphometrics_r(input_directory=object_path, output_directory=working_directory, PC_axis1="1", PC_axis2="4", img_height_pca=1000, img_width_pca=1000)

In [ ]:
#For running kmeans
object_path=os.path.join(working_directory,"efourier_results","pca_fourier.rds")
run_kmeans_efourier_r(pca_objects_path=object_path, output_directory=working_directory,max_clusters=10, img_height_pca=1000, img_width_pca=1000, plot_xlim=250, plot_ylim=250)

## PCA Pixel-based

In [ ]:
#For run Pixel-Based PCA analysis
process_images_and_perform_pca(directory=input_masks, working_directory=working_directory, n_components=50, k_max=10, std_multiplier=3)
